In [1]:
import pyten as ptn
import numpy as np
import time
import os
import matplotlib.pyplot as plt

## Find the groundstate and low-lying excited states of Ising and Heisenberg.
Code largely by Sebastian Paeckel and Sam Mardazad

In [2]:
# create DMRG configuration object
conf = ptn.dmrg.DMRGConfig()
# give us a list to add stages
stages = []
# add first stage
stages.append(ptn.dmrg.DMRGStage())
stages[0].trunc.maxStates = 16
stages[0].convergenceMaxSweeps = 30
stages[0].trunc.weight = 1e-6
# add second stage
stages.append(ptn.dmrg.DMRGStage())
stages[1].trunc.maxStates = 32
stages[1].convergenceMaxSweeps = 20
stages[1].trunc.weight = 1e-6
# add third stage
stages.append(ptn.dmrg.DMRGStage())
stages[2].trunc.maxStates = 64
stages[2].convergenceMaxSweeps = 20
stages[2].convMinEnergyDiff = 1e-07  
stages[2].trunc.weight = 1e-8
#
stages.append(ptn.dmrg.DMRGStage())
stages[3].trunc.maxStates = 128
stages[3].convergenceMaxSweeps = 20
stages[3].convMinEnergyDiff = 1e-12 
stages[3].trunc.weight = 1e-10
#
stages.append(ptn.dmrg.DMRGStage())
stages[4].trunc.maxStates = 256
stages[4].convergenceMaxSweeps = 10
stages[4].convMinEnergyDiff = 1e-12 
stages[4].trunc.weight = 1e-12
#
stages.append(ptn.dmrg.DMRGStage())
stages[5].trunc.maxStates = 512
stages[5].convergenceMaxSweeps = 10
stages[5].convMinEnergyDiff = 1e-12  
stages[5].trunc.weight = 1e-14
# assign stages to DMRG configuration object
conf.stages = stages

In [3]:
j_list = np.arange(-2., 2.1, 0.1)
j_list = np.array(j_list)

In [4]:
# L: number of sites
# model: 'ising' or 'heisenberg'
# spin: 0.5 or 1
# states_number: 0 for just the groundstate, more for excited states 
def calc_states(L=32, model='ising', spin=0.5, states_number = 6):
    h = 1

    # containers to save to
    energies = []
    timings = []
    
    fname = model + str(L)
    if spin == 1.:
        folder = './data_spinOne/'
    elif spin == 0.5:
        folder = './data_spinHalf/'
    for savepath in [folder]:
        if not os.path.exists(savepath):
            os.makedirs(savepath)

    ''' loop over J values '''
    for j_num, J in enumerate(j_list):
        print('Start ' + fname, '| spin', np.round(spin, 2), '| J', np.round(J, 2))

        ''' lattice '''
        if model == 'ising':
            lattice = ptn.mp.lat.nil.genSpinLattice(L, spin)
        elif model == 'heisenberg':
            # uses U(1) symmetry # only with spin 1.0?
            lattice = ptn.mp.lat.u1.genSpinLattice(L, spin)

        ''' Hamiltonian '''
        hamiltonian = []
        if model == 'ising':
            if spin == 1.:
                for i in range(0, L-1):
                    hamiltonian.append( - J * lattice.get("sz",i)*lattice.get("sz",i+1) )  # zz
                    hamiltonian.append( - h *  lattice.get("sx",i) )  # x
            elif spin == 0.5:
                for i in range(0, L-1):
                    hamiltonian.append( - 4 * J * lattice.get("sz",i)*lattice.get("sz",i+1) )  # zz
                    hamiltonian.append( - 2 * h  *  lattice.get("sx",i) )  # x
        elif model == 'heisenberg':
            if spin == 1.:
                for i in range(0, L-1):
                    H_u1.append( -h * ( lat_u1.get("sp",i)*lat_u1.get("sm",i+1) + lat_u1.get("sm",i)*lat_u1.get("sp",i+1) ))  # ~ (xx + yy)
                    H_u1.append( -2 * J  * lat_u1.get("sz",i)*lat_u1.get("sz",i+1))
            elif spin == 0.5:
                print('Does heisenberg with U(1) work with Spin 0.5?')
                break
        # add H to lattice
        hamiltonian = ptn.mp.addLog(hamiltonian)
        lattice.add("H", "S=" + str(spin) + " " + model + " Hamiltonian with h={} and J={}".format(h,J), hamiltonian, True)

        
        ''' states '''
        energies_states = []
        energies_states.append(J)
        timings_states = 0
        prev_states = []
        for state in range(states_number):
            Hs = []
            Hs.append(hamiltonian)
            # create initial state (second argument is target symmetry sector)
            initial_state = ptn.mp.generateCompleteState(lattice, "0.0")
            # dmrg and logging
            dmrg       = ptn.mp.dmrg.PDMRG(initial_state, Hs, conf, prev_states)
            Es_state      = np.array([])
            timings_state = np.array([])
            states_state = []  # MPS
            tstart = time.perf_counter();
            # stages
            for m in conf.stages:
                # run stage until either convergence is met or max. number of sweeps
                states_state.append(dmrg.run())
                timings_state = np.append(timings_state, time.perf_counter()-tstart)
                Es_state = np.append(Es_state, ptn.mp.expectation(states_state[-1], lattice.get("H")))

            energies_states.append(Es_state[-1])
            timings_states += np.sum(timings_state)
            prev_states.append(states_state[-1])

        # save 
        energies.append(energies_states)
        timings.append([J, timings_states])

        # save to file
        np.savetxt(folder + fname + '_energy' + '.csv', energies, delimiter=',')
        np.savetxt(folder + fname + '_time' + '.csv', timings, delimiter=',')
        
    print('finished ' + fname + '\n')

In [ ]:
calc_states(L=4, model='ising', spin=1.)
calc_states(L=8, model='ising', spin=1.)
calc_states(L=16, model='ising', spin=1.)
calc_states(L=32, model='ising', spin=1.)


Start ising4 | spin 1.0 | J -2.0
Start ising4 | spin 1.0 | J -1.9
Start ising4 | spin 1.0 | J -1.8
Start ising4 | spin 1.0 | J -1.7
Start ising4 | spin 1.0 | J -1.6
Start ising4 | spin 1.0 | J -1.5
Start ising4 | spin 1.0 | J -1.4
Start ising4 | spin 1.0 | J -1.3
Start ising4 | spin 1.0 | J -1.2
Start ising4 | spin 1.0 | J -1.1
Start ising4 | spin 1.0 | J -1.0
Start ising4 | spin 1.0 | J -0.9
Start ising4 | spin 1.0 | J -0.8
Start ising4 | spin 1.0 | J -0.7
Start ising4 | spin 1.0 | J -0.6
Start ising4 | spin 1.0 | J -0.5
Start ising4 | spin 1.0 | J -0.4
Start ising4 | spin 1.0 | J -0.3
Start ising4 | spin 1.0 | J -0.2
Start ising4 | spin 1.0 | J -0.1
Start ising4 | spin 1.0 | J 0.0
Start ising4 | spin 1.0 | J 0.1
Start ising4 | spin 1.0 | J 0.2
Start ising4 | spin 1.0 | J 0.3
Start ising4 | spin 1.0 | J 0.4
Start ising4 | spin 1.0 | J 0.5
Start ising4 | spin 1.0 | J 0.6
Start ising4 | spin 1.0 | J 0.7
Start ising4 | spin 1.0 | J 0.8
Start ising4 | spin 1.0 | J 0.9
Start ising4 | spin 

In [ ]:
#calc_states(L=4, model='heisenberg', spin=0.5)
#calc_states(L=8, model='heisenberg', spin=0.5)
#calc_states(L=16, model='heisenberg', spin=0.5)
#calc_states(L=32, model='heisenberg', spin=0.5)